<a href="https://colab.research.google.com/github/MTracchia/Laboratorio-de-Datos/blob/main/Clases/Web%20scraping/Contenido_est%C3%A1tico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Trabajando con el contenido estático de una página

Vamos a comenzar trabajando en forma análoga a lo que hacíamos con las APIs, haciendo un simple request, nada más que ahora vamos a conseguir un código HTML en vez de datos bien formateadas en json o xml.

Al hacer un request estamos sacando un foto de la página: su contenido es completamente estático. Por estático nos referimos a que no vamos a tener habilitadas opciones como *scrollear* o bien clickear algún que otro botón. Esto último es importante porque muchas veces hay contenido que solo se puede acceder a través de dichos métodos. Trabajar con contenido dinámico lo vamos a ver aparte de esta notebook (cuando trabajemos, por ejemplo, con la librería *selenium*). 

Dado que sacamos una foto de la página, recibimos un código que vamos a tener que *parsearlo*, es decir, convertirlo en un objeto manipulable en el que sea fácil navegar. Esto lo hacemos con el módulo *BeautifulSoup*.



In [ ]:
# Importamos las librerías que vamos a utilizar
import requests
from bs4 import BeautifulSoup as BS 

Comencemos como prueba con el *homepage* del diario La Nación. Recordar que una vez que hacemos el request ya dejamos de interactuar con la página:

In [ ]:
# Hacemos un request a la página de La Nación
response = requests.get("https://www.lanacion.com.ar")

# Vemos el contenido que nos devolvió
print(response.content)

b'<!DOCTYPE html>\r\n<html lang="es">\r\n<head>\r\n\r\n\r\n<script>\r\n    \r\n    \r\n    var varnish = [\'\'];\r\n\r\n    Array.from = Array.from || (function () { var a = Object.prototype.toString, b = function (f) { return \'function\' == typeof f || \'[object Function]\' === a.call(f) }, c = function (f) { var g = +f; return isNaN(g) ? 0 : 0 != g && isFinite(g) ? (0 < g ? 1 : -1) * Math.floor(Math.abs(g)) : g }, d = Math.pow(2, 53) - 1, e = function (f) { var g = c(f); return Math.min(Math.max(g, 0), d) }; return function (g) { var h = this, i = Object(g); if (null == g) throw new TypeError(\'Array.from requires an array-like object - not null or undefined\'); var l, j = 1 < arguments.length ? arguments[1] : void 0; if (\'undefined\' != typeof j) { if (!b(j)) throw new TypeError(\'Array.from: when provided, the second argument must be a function\'); 2 < arguments.length && (l = arguments[2]) } for (var p, m = e(i.length), n = b(h) ? Object(new h(m)) : Array(m), o = 0; o < m;)p = i

Listo! La información que nos dió el request es todo con lo que contamos. Ahora no queda otra que arremangarse y trabajar con esto. Para eso, *BeautifulSoup* nos da bastantes facilidades. Primero creamos la sopa:

In [ ]:
# soup es un objeto manipulable creado a partir del contenido de la página y BeautifulSoup
soup = BS(response.content)

soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="es">
<head>
<script>
    
    
    var varnish = [''];

    Array.from = Array.from || (function () { var a = Object.prototype.toString, b = function (f) { return 'function' == typeof f || '[object Function]' === a.call(f) }, c = function (f) { var g = +f; return isNaN(g) ? 0 : 0 != g && isFinite(g) ? (0 < g ? 1 : -1) * Math.floor(Math.abs(g)) : g }, d = Math.pow(2, 53) - 1, e = function (f) { var g = c(f); return Math.min(Math.max(g, 0), d) }; return function (g) { var h = this, i = Object(g); if (null == g) throw new TypeError('Array.from requires an array-like object - not null or undefined'); var l, j = 1 < arguments.length ? arguments[1] : void 0; if ('undefined' != typeof j) { if (!b(j)) throw new TypeError('Array.from: when provided, the second argument must be a function'); 2 < arguments.length && (l = arguments[2]) } for (var p, m = e(i.length), n = b(h) ? Object(new h(m)) : Array(m), o = 0; o < m;)p = i[o], n[o] = j ? 

Ahora todo lo que queda es encontrar los elementos que nos interese. 
¿Cómo los identificamos? Es un trabajo de ida y vuelta, y prueba y error, así que a no frustrarse si no sale de entrada. Los pasos a seguir son:
- Ir a la página que queremos scrappear y con el botón derecho del mouse poner "inspeccionar elemento" sobre el sector de la página que nos interese. 
- Luego podemos identificarlo por su *tag* o por algún atributo que nos permita identificarlo mejor, como por ejemplo, la clase (*class*).
- A veces puede ser más conveniente no apuntarle al elemento que nos interesa sino a un nodo superior (que lo contenga) y a partir de ahí navegar hacia dentro. 

Investigando la página del diario La Nación encontramos que los recuadros correspondientes a los títulos y enlaces de las principales notas están dentro de bloques con atributo:
~~~
<... class = "com-title">
~~~
Como observamos que el tag puede ser *h1* o *h2*, vamos a tratar de no especificar el *tag* para no perdernos nada. Buscamos entonces todos los elementos que cumplan el requisito de la clase sin especificar el *tag*:

In [ ]:
# Creamos una lista con los elementos cuya clase = "com-title" con cualquier tag
elements = soup.find_all(attrs = {"class": "com-title"})

Vemos en qué consiste la lista de elementos: 

In [ ]:
print(elements)

[<h1 class="com-title">
<a href="https://www.lanacion.com.ar/sociedad/coronavirus-en-la-argentina-reportan-641-muertos-y-35355-nuevos-casos-nid01062021/" title="Reportan 641 muertos y 35.355 nuevos casos de Covid">
<!--Crear Parcial para Volanta-->
<em class="com-volanta">Pandemia.</em>
        Reportan 641 muertos y 35.355 nuevos casos de Covid
    </a>
</h1>, <h2 class="com-title">
<a href="https://www.lanacion.com.ar/el-mundo/la-trama-mafiosa-detras-de-la-venta-de-terrenos-en-un-exclusivo-barrio-de-jose-ignacio-nid01062021/" title="La trama detrás de la venta de terrenos de argentinos en José Ignacio">
<!--Crear Parcial para Volanta-->
<em class="com-volanta">Mafia.</em>
        La trama detrás de la venta de terrenos de argentinos en José Ignacio
    </a>
</h2>, <h2 class="com-title">
<a href="https://www.lanacion.com.ar/politica/murio-el-consultor-y-analista-politico-jorge-giacobbe-nid01062021/" title="Murió el consultor Jorge Giacobbe">
<!--Crear Parcial para Volanta-->
<em class

Cada elemento tiene por ejemplo enlace a la página (*href*) de la nota que va a estar contenido dentro de un *tag < a >*.
Entonces lo que hacemos es iterar sobre todos los elementos y encontrar el *tag a* y pedirle el valor del atributo *href*:

In [ ]:
# Iteramos sobre los elementos
# Hacemos un try ... except ... por si alguno de los elementos no tiene enlace 
# (puede ser que estemos agarrando más cosas de las que nos interesa)
urls = []
for element in elements:
  try:
    urls.append(element.find('a')['href'])
  except:
    pass

print(urls)

['https://www.lanacion.com.ar/sociedad/coronavirus-en-la-argentina-reportan-641-muertos-y-35355-nuevos-casos-nid01062021/', 'https://www.lanacion.com.ar/el-mundo/la-trama-mafiosa-detras-de-la-venta-de-terrenos-en-un-exclusivo-barrio-de-jose-ignacio-nid01062021/', 'https://www.lanacion.com.ar/politica/murio-el-consultor-y-analista-politico-jorge-giacobbe-nid01062021/', 'https://www.lanacion.com.ar/seguridad/dumbo-el-capo-narco-que-se-mudo-a-martinez-y-dejo-el-mate-tibio-para-poder-escapar-de-la-policia-nid01062021/', 'https://www.lanacion.com.ar/sociedad/covid-19-avanza-en-el-instituto-leloir-otra-vacuna-argentina-nid01062021/', 'https://www.lanacion.com.ar/deportes/futbol/la-generosa-despedida-de-sergio-aguero-con-los-empleados-del-manchester-city-mas-de-60-relojes-nid01062021/']


In [ ]:
print(len(urls))

6


Ya tenemos enlaces de páginas (¿son todos los que vemos cuando abrimos la página? ¿No habrá contenido dinámico que nos estamos perdiendo?).

### Scrappeo de la paǵina de un artículo

Veamos si podemos extraer contenido de las notas cuyos enlaces extraímos. La idea es la misma: ir a la página e inspeccionar los elementos que nos interese. Cuando las páginas corresponden a la misma plataforma es usual que la información se encuentre en el mismo lugar, por lo que aprendamos de una sola paǵina podemos quizás extrapolarlo a otras.

Agarremos un *url* (por ejemplo, alguno de los de arriba) y generemos la sopa:

In [ ]:
# Url de interés
url = 'https://www.lanacion.com.ar/sociedad/coronavirus-en-la-argentina-reportan-641-muertos-y-35355-nuevos-casos-nid01062021/'

# Hacemos un request a dicha página
response = requests.get(url)

# Creamos la sopa para manipular
soup = BS(response.content)

Identificando dónde está el título de la nota (eso de vuelta, inspeccionando el elemento asociado), podemos extraer el texto:

In [ ]:
# Buscamos el elemento cuya clase indentificamos como correspondiente al título de la nota
title = soup.find(attrs = {"class": "com-title --threexl"})
print(title.text)

Coronavirus en la Argentina: reportan 641 muertos y 35.355 nuevos casos


Fecha de la nota: 

In [ ]:
date = soup.find(attrs = {"class": "com-date --twoxs"}).text
print(date)

1 de junio de 2021


O, urgando un poco más en el código HTML, podemos extraer la fecha mejor formateada (notar otra manera de buscar cosas con BS):

In [ ]:
# Buscamos un tag del tipo "meta", cuyo atributo "property" tiene valor "article:published_time"
# A este tag le sacamos el valor que toma el atributo "content"
datetime = soup.find("meta", attrs = {"property": "article:published_time"})['content']
print(datetime)

2021-06-01T20:17:28.924Z


Veamos la descripción de la nota, que podemos encontrarla en:

In [ ]:
description = soup.find(attrs = {"class": "com-subhead --bajada --m-xs"}).text
print(description)

Los fallecidos en el país se acercan a los 79.000, mientras los internados en Unidades de Terapia Intensiva continúan en alza


Por último veamos el cuerpo de la nota. Encontramos un nodo padre del tipo *section* cuya clase es *cuerpo__nota* (notar que tenemos que escribir todo exactamente como lo vemos en el código de la página. Cualquier error de tipeo va a resultar en que no nos devuelva nada):

In [ ]:
# Buscamos un tag del tipo "section", cuyo atributo "class" tiene valor "cuerpo__nota"
body = soup.find("section", attrs = {"class": "cuerpo__nota"})

Usualmente, los párrafos están dentro de tags denominados *p*. Por lo que buscamos todos estos dentro del bloque asociado al cuerpo de la nota: 

In [ ]:
# Párrafos usualmente dentro de tags p
paragraphs = body.find_all('p')

print(paragraphs)

[<p class="com-paragraph --capital --s">El Gobierno informó que en las últimas 24 horas se reportaron<strong> 35.355 nuevos casos de Covid y 641 fallecidos. </strong>De esta manera, el país ya cuenta con 78.733 muertos.</p>, <p class="com-paragraph --s">Con los 35.355 casos positivos de coronavirus Covid-19, según los datos publicados en la Sala de Situación Coronavirus online del Ministerio de Salud de la Nación, <strong>las personas que dieron positivo por el virus SARS-CoV-2 en el país desde el inicio de la pandemia son 3.817.139. </strong>De este número, ya se recuperaron <strong>3.381.337 y los que aún cursan la enfermedad son 357.069.</strong></p>, <p class="com-paragraph --s">Las provincias que reportaron contagios fueron:</p>, <p class="com-paragraph --s">Entre los muertos ingresados en el Sistema Naciona de Vigilancia en Salud (SNVS), <strong>seis personas residentes en la provincia de Buenos Aires, una en la ciudad de Buenos Aires, una en Misiones, una en Mendoza y una en Ent

In [ ]:
# Iteramos para todo párrafo y vemos el texto contenido
# Podemos hacer una lista y concatenar todo con un salto de línea
texts = []
for p in paragraphs:
  texts.append(p.text)

body_text = '\n'.join(texts)

print(body_text)

El Gobierno informó que en las últimas 24 horas se reportaron 35.355 nuevos casos de Covid y 641 fallecidos. De esta manera, el país ya cuenta con 78.733 muertos.
Con los 35.355 casos positivos de coronavirus Covid-19, según los datos publicados en la Sala de Situación Coronavirus online del Ministerio de Salud de la Nación, las personas que dieron positivo por el virus SARS-CoV-2 en el país desde el inicio de la pandemia son 3.817.139. De este número, ya se recuperaron 3.381.337 y los que aún cursan la enfermedad son 357.069.
Las provincias que reportaron contagios fueron:
Entre los muertos ingresados en el Sistema Naciona de Vigilancia en Salud (SNVS), seis personas residentes en la provincia de Buenos Aires, una en la ciudad de Buenos Aires, una en Misiones, una en Mendoza y una en Entre Ríos no presentaban datos de género. Del resto, 367 eran hombres que vivían en:
En tanto, las mujeres eran 264 y vivían en:
Por otra parte, ayer eran 7417 las personas que cursaban la enfermedad en 

### Iteración sobre varios urls

Una de las cosas interesantes del scrappeo es poder iterar sobre varias página y extraer información en forma sistemática. Extraigamos el título y la descripción para un conjunto de urls. Básicamente es hacer lo que hicimos para una nota varias veces:

In [ ]:
urls

['https://www.lanacion.com.ar/sociedad/coronavirus-en-la-argentina-reportan-641-muertos-y-35355-nuevos-casos-nid01062021/',
 'https://www.lanacion.com.ar/el-mundo/la-trama-mafiosa-detras-de-la-venta-de-terrenos-en-un-exclusivo-barrio-de-jose-ignacio-nid01062021/',
 'https://www.lanacion.com.ar/politica/murio-el-consultor-y-analista-politico-jorge-giacobbe-nid01062021/',
 'https://www.lanacion.com.ar/seguridad/dumbo-el-capo-narco-que-se-mudo-a-martinez-y-dejo-el-mate-tibio-para-poder-escapar-de-la-policia-nid01062021/',
 'https://www.lanacion.com.ar/sociedad/covid-19-avanza-en-el-instituto-leloir-otra-vacuna-argentina-nid01062021/',
 'https://www.lanacion.com.ar/deportes/futbol/la-generosa-despedida-de-sergio-aguero-con-los-empleados-del-manchester-city-mas-de-60-relojes-nid01062021/']

In [ ]:
# Barremos en el listado de urls 
# Le ponemos un try-except para ignorar errores
for url in urls: 
  
  try:
  
    # Hacemos request al url actual
    response = requests.get(url)

    # Creamos la sopa
    soup = BS(response.content)

    # Identificamos el título
    title = soup.find(attrs = {"class": "com-title --threexl"}).text

    # Identificamos la descripción
    description = soup.find(attrs = {"class": "com-subhead --bajada --m-xs"}).text

    # Printeamos ambas y dejamos un espacio entre nota y nota
    print(title)
    print(description)
    print('\n\n')
  
  except:
    pass

Coronavirus en la Argentina: reportan 641 muertos y 35.355 nuevos casos
Los fallecidos en el país se acercan a los 79.000, mientras los internados en Unidades de Terapia Intensiva continúan en alza



La trama mafiosa detrás de la venta de terrenos de argentinos en un exclusivo barrio de José Ignacio
Un escribano y un operador inmobiliario, junto con varios cómplices, se hacían de valiosas propiedades que pertenecían a personas fallecidas, entre ellas argentinas, y las vendía a un tercio de su valor



Murió el consultor y analista político Jorge Giacobbe
Era el presidente de Giacobbe & Asociados Opinión Pública S.A. desde el año 2000; había estudiado Economía en la Universidad de Morón y Derecho en la Universidad de Buenos Aires



Dumbo: el capo narco que se mudó a Martínez y dejó el mate tibio para poder escapar de la policía
Raúl Martín Maylli Rivera, acusado de liderar una organización criminal en el barrio Padre Mugica, de Villa Lugano, volvió a fugarse cuando fueron a buscarlo a

### Algunas conclusiones que sacamos de aquí

Vimos un ejemplo concreto de cómo extraer contenido de ciertas páginas. Algunas aclaraciones:

- No suele ser lo mejor hacer esto desde un colab. Lo ideal es tener algún bot creado localmente. 
- Si vamos a iterar sobre varias páginas tener presente recomendaciones tales como no hacer demasiadas requests en un período de tiempo, que podrían resultar en prohibirnos el acceso a diferentes páginas.
- Lo que vimos acá no maneja contenido dinámico: en particular, en el ejemplo que vimos la información va emergiendo a medida que scrolleamos. 

El manejo de contenido dinámico es con el lenguaje javascript y podemos manejarlo con la librería *selenium*. Esto lo vamos a hacer corriendo el siguiente script por fuera del  (en spyder por ejemplo o desde la terminal *python script.py*). Con este podemos obtener muchos más enlaces y luego scrappear cada uno de ellos:

~~~
# -*- coding: utf-8 -*-
"""
El siguiente código es complemento de la notebook sobre el manejo 
de contenido estático de una paǵina de la clase de web scrapping. 
La idea aquí es que en muchas páginas la información se muestra en forma dinámica, 
por lo que vamos a necesitar manejar cosas de javascript. 
Esto lo podemos hacer con selenium, 
que simula un navegador web como si fuera un usuario más.
"""
# ---------- Importación de librerías ----------- #

# Importamos algunos módulos de selenium que nos van a servir
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Importamos la libreŕía time para esperar cierto tiempo
import time 

# ---------------------------------------------- #

# ------------ Creación del navegador --------- #

# Para que selenium ande hay que descargar un driver, que depende de cada 
# navegador y sistema operativo, y se descarga en la página de selenium:
# https://selenium-python.readthedocs.io/installation.html#drivers

# Para especificar donde está se lo pasamos 
# como argumento cuando inicializamos el navegador. 
# Para linux, hay que convertirlo en ejecutable chmod +x driver
# Para mas detalles:
# https://stackoverflow.com/questions/42478591/python-selenium-chrome-webdriver

PATH_DRIVER = "..." # Cambiar aquí dónde está

# Creamos un navegador tipo Chrome. 
# Podemos decirle que navegue en forma explícita o ímplícita.
# La segunda lo logramos descomentando la línea "--headless".

chrome_options = Options()
chrome_options.add_argument("--headless")

# Creación del navegador
driver = webdriver.Chrome(executable_path = PATH_DRIVER, options = chrome_options)

# -------------------------------------------- #

# --------- Visitamos la página -------------- #

# Visitamos la página y esperamos 5 segundos a que todo cargue bien
driver.get("https://www.lanacion.com.ar")
time.sleep(5)

# Acá manejamos algo dinámico: 
# Dado que en La Nación el contenido aparece scrolleando vamos a hacer eso. 
# Todos las líneas de código en javascript se pueden ejecutar 
# con ".execute_script" e insertando el código correspondiente allí.
# Lo que vamos a hacer acá es scrollear varias veces 
# hasta el final de la página y esperar un poco a que se cargue bien la página

for iteration in range(6):
	driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
	time.sleep(2)

# Una vez que dejamos scrollear identicamos todos los bloques asociados a las notas (ver notebook). 
# Cómo se busca en selenium? https://selenium-python.readthedocs.io/locating-elements.html

elements = driver.find_elements_by_class_name("com-title")

# Ahora guardamos los enlaces de cada elemento
hrefs = []
for element in elements:
    try:
	href = element.find_element_by_tag_name("a").get_attribute("href")
	hrefs.append(href)
    except:
        pass

# Guardamos los enlaces en un archivo
fp = open('links.txt','w')
for href in hrefs:
	fp.write(href + '\n')
fp.close()

# Finalmente cerramos todas las sesiones de navegación que abrimos. 
# Son necesarias ambas líneas (una cierra la pestaña, la otra toda el navegador)

driver.close()
driver.quit()
~~~